In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib import gridspec

import numpy as np
import scipy.sparse as sp
import numpy.linalg as la

In [16]:
def add_relative_to_current_source_file_path_to_sys_path(relpath):
    import os, sys, inspect
    path = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0],relpath)))
    if path not in sys.path:
        sys.path.insert(0,path)

add_relative_to_current_source_file_path_to_sys_path("../../shims/toric")   

import toric
import toric.samples
import toric.shims

#
# Helper functions for the toric libary
#

def vector3_to_str(v):
    return "(%.2f, %.2f, %.2f)" % (v.x(), v.y(), v.z())

def quaternion_to_str(q):
    return "(w,x,y,z): (%.2f, %.2f, %.2f, %.2f)" % (q.w(), q.x(), q.y(), q.z())

def vector3_to_np3(v):
    return np.array([v.x(),v.y(),v.z()])

def np3_to_vector3(v):
    return toric.Vector3(v[0],v[1],v[2])

def slerp(p0, p1, t):
    omega = np.arccos(np.dot(p0/np.linalg.norm(p0), p1/np.linalg.norm(p1)))
    so = np.sin(omega)
    return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1
    
def fovX_from_fovY(fovY, aspect):
    tanY = np.tan(fovY.valueRadians() / 2.0)
    tanX = tanY * aspect;
    fovX = np.arctan(tanX)
    return toric.RadianPi(fovX)

def drawSetup(fudgeX=10, fudgeY_pos=6,fudgeY_neg=8):
    pylab.rcParams['figure.figsize'] = 8, 8

    plt.axes().set_aspect('equal')
    fig = plt.gcf()
    
    # Set up plot size
    plt.axes().set_xlim((np.min([PA.x(), PB.x()])-fudgeX,np.max([PA.x(), PB.x()])+fudgeX))
    plt.axes().set_ylim((np.min([PA.y(), PB.y()])-fudgeY_neg,np.max([PA.y(), PB.y()])+fudgeY_pos))

    # Draw People Positions
    plt.scatter([PA.x(), PB.x()],[PA.y(), PB.y()],c="red",linewidths=0)
    line_AB = plt.Line2D([PA.x(), PB.x()],[PA.y(), PB.y()], c="black",alpha=0.3)
    fig.gca().add_artist(line_AB)

    # Draw Circles    
    circle_PA=plt.Circle((PA.x(),PA.y()),min_dist,color='g',alpha=0.5)
    circle_PB=plt.Circle((PB.x(),PB.y()),min_dist,color='g',alpha=0.5)
    
    fig.gca().add_artist(circle_PA)
    fig.gca().add_artist(circle_PB)

    plt.annotate(s="A", xy=(PA.x(),PA.y()),xytext=(3,4),textcoords="offset points")
    plt.annotate(s="B", xy=(PB.x(),PB.y()),xytext=(3,4),textcoords="offset points")
    
    # Draw Camera positions
    plt.scatter([C.x()],[C.y()],c="blue",linewidths=0)
    plt.annotate(s="C", xy=(C.x(), C.y()),xytext=(3,4),textcoords="offset points")

In [63]:
# Conversion from world space to camera screen space (from Unity: z,x,-y)

# camera specs (position, forward, up, fov)
C = toric.Vector3(-92.9,-39.6,-2.1)
forward = toric.Vector3(-1.0,0.0,0.0)
up = toric.Vector3(0.0,0.0,-1.0)
aspect = 16.0/9.0
fovY = toric.RadianPi(0.8726646)
fovX = fovX_from_fovY(fovY, aspect)

# scene specs
PA = toric.Vector3(-98.2,-40.6,-1.6)     # person A position
PB = toric.Vector3(-98.2,-38.6,-1.6)     # person B position

In [66]:
# http://www.opengl-tutorial.org/beginners-tutorials/tutorial-3-matrices/
# http://stackoverflow.com/questions/21830340/understanding-glmlookat
def lookAt(camPos, camLookAt, camUp):
    z = camPos - camLookAt
    z = z / la.norm(z) # forward
    y = camUp # up
    x = np.cross(y, z) # right
    y = np.cross(z, x)
    x = x / la.norm(x)
    y = y / la.norm(y)
    v_matrix = np.matrix([[x[0], y[0], z[0], -np.dot(x, camPos)], 
                          [x[1], y[1], z[1], -np.dot(y, camPos)],
                          [x[2], y[2], z[2], -np.dot(z, camPos)],
                          [0.0, 0.0, 0.0, 1.0]])
    return v_matrix
    
def view_matrix(camPos, camLookAt, camUp):
    return lookAt(camPos, camLookAt, camUp)

look = vector3_to_np3(C) + vector3_to_np3(forward)
view = view_matrix(vector3_to_np3(C), look, vector3_to_np3(up))
print "view"
print view

view
[[  0.    0.    1.  -39.6]
 [ -1.    0.    0.   -2.1]
 [  0.   -1.    0.   92.9]
 [  0.    0.    0.    1. ]]


In [67]:
# https://www.opengl.org/sdk/docs/man2/xhtml/gluPerspective.xml
# https://unspecified.wordpress.com/2012/06/21/calculating-the-gluperspective-matrix-and-other-opengl-matrix-maths/
def perspective_matrix(fovY, aspect, near, far):
    f = 1.0 / np.tan (fovY / 2.0)
    p_matrix = np.matrix([[f/aspect, 0.0, 0.0, 0.0],
                          [0.0, f, 0.0, 0.0],
                          [0.0, 0.0, (far+near)/(near-far), (2.0*far*near)/(near-far)], 
                          [0.0, 0.0, -1.0, 0.0]])
    return p_matrix

proj = perspective_matrix(fovY.valueRadians(), aspect, 0.1, 100.0)
print "proj"
print proj

model = np.identity(4)
print "model"
print model

mvp = proj * view * model
print "mvp"
print mvp

proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]
model
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
mvp
[[  0.           0.           1.20628518 -47.76889328]
 [ -2.14450699   0.           0.          -4.50346469]
 [  0.           1.002002     0.         -93.28618619]
 [  0.           1.           0.         -92.9       ]]


In [74]:
# test center point
homogeneous_center_world = np.array([[look[0]], 
                               [look[1]],
                               [look[2]], 
                               [1.0]])
screenA_vec4 = mvp * homogeneous_center_world
print screenA_vec4
test_view = view * homogeneous_center_world
print test_view

screenA_vec3 = np.array([screenA_vec4[0]/screenA_vec4[3], screenA_vec4[1]/screenA_vec4[3], screenA_vec4[2]/screenA_vec4[3]])
print screenA_vec3

[[ -50.30209216]
 [ 196.86574198]
 [-132.96546547]
 [-132.5       ]]
[[ -41.7]
 [  91.8]
 [ 132.5]
 [   1. ]]
[[[ 0.37963843]]

 [[-1.48577918]]

 [[ 1.00351295]]]


In [73]:
screenA_vec4 = mvp * np.array([[PA.x()], 
                               [PA.y()],
                               [PA.z()], 
                               [1.0]])
print screenA_vec4

screenA_vec3 = np.array([screenA_vec4[0]/screenA_vec4[3], screenA_vec4[1]/screenA_vec4[3], screenA_vec4[2]/screenA_vec4[3]])
print screenA_vec3

[[ -49.69894957]
 [ 206.08712205]
 [-133.96746747]
 [-133.5       ]]
[[[ 0.37227678]]

 [[-1.54372376]]

 [[ 1.00350163]]]
